# Tyler Cady 12/10/2017

# Show the offenders that have “Pokemon” in the description. Show only the offender, license plate, and the violation description, not the keys.

## Import pyspark modules

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql import Row


## Create a Spark SQL Session

In [20]:
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()

try:
    conf = SparkConf().setMaster("local[*]")
   
except ValueError:
    print('Spark Context already running')

## Show the violations that have “Pokemon” in the description. Show only the license plate and the description (not the keys)

### Create Schema for violations file

In [21]:
def parse_violations(line):
    fields = line.split(',')
    return Row(violationId=int(fields[0]), vdate=fields[1], town=fields[2], road=fields[3], \
               vdescription=fields[4], officer=fields[5], licensePlate=fields[6], offenderId = int(fields[7]) )

### Create Schema for violations file

In [22]:
def parse_violations(line):
    fields = line.split(',')
    return Row(violationId=int(fields[0]), vdate=fields[1], town=fields[2], road=fields[3], \
               vdescription=fields[4], officer=fields[5], licensePlate=fields[6], offenderId = int(fields[7]) )

### Create Schema for offender file

In [23]:
def parse_offenders(line):
    fields = line.split(',')
    return Row(offId=int(fields[0]), firstName=fields[1], lastName=fields[2], address=fields[3], \
               city=fields[4], state=fields[5], zip=fields[6], licenseNo=fields[7], licenseState=fields[8], \
               birthday = fields[9], height = fields[10], eyeColor = fields[11])

### Read violations file and assign column name

In [24]:
vioRDD = spark.sparkContext.textFile("violations.csv")
vioRDD.take(3)

['1,6/10/2015,Tacoma,815 Fire Access Road,Texting while driving,1,CJA 910,1',
 '2,7/28/2015,Bellevue,3379 Platinum Drive,Playing Pokemon while driving,1,UBT 465,2',
 '3,11/8/2015,Seattle,411 Lighthouse Drive,Improper lane change,1,CDG 467,3']

In [25]:
svioRDD = vioRDD.map(parse_violations)

### Read offenders fileand assign column name

In [26]:
offRDD = spark.sparkContext.textFile("offenders.csv")
offRDD.take(3)

['1,David,Bush,4097 Willison Street,Minneapolis,MN,55415,474-40-6641,WA,9/6/1943,"5\' 8""",Blue',
 '2,Kristie,Carpenter,1393 Elk Avenue,Britton,MI,49229,373-28-8156,WA,6/28/1989,"5\' 5""",Green',
 '3,Roy,Skinner,722 James Street,Rochester,NY,14611,070-60-4251,WA,2/19/1961,"5\' 8""",Brown']

In [27]:
soffRDD = offRDD.map(parse_offenders)

### Register data frame as table

In [28]:
schemaVio = spark.createDataFrame(svioRDD).cache()
schemaVio.createOrReplaceTempView('violations')

In [29]:
schemaOff = spark.createDataFrame(soffRDD).cache()
schemaOff.createOrReplaceTempView('offenders')

### Show license plate and description

In [30]:
allViolations = spark.sql("select firstName, lastName, licensePlate, vdescription from violations v \
                          join offenders o on v.offenderId = o.offID \
                          where vdescription like '%Pokemon%'")
allViolations.take(3)

[Row(firstName='Alan', lastName='Arbuckle', licensePlate='XKM 123', vdescription='Playing Pokemon while driving'),
 Row(firstName='Thomas', lastName='Watkins', licensePlate='DLF 145', vdescription='Playing Pokemon while driving'),
 Row(firstName='Raymond', lastName='Llamas', licensePlate='VCW 805', vdescription='Playing Pokemon while driving')]

### Iterate through dataframe and show only values, not keys

In [31]:
vio = allViolations.collect()
for i in vio:
    print(i[0], i[1], i[2], i[3])
    

Kristie Carpenter UBT 465 Playing Pokemon while driving
Raymond Llamas VCW 805 Playing Pokemon while driving
Thomas Watkins DLF 145 Playing Pokemon while driving
Alan Arbuckle XKM 123 Playing Pokemon while driving
Jessica Lozano DCY 965 Playing Pokemon while driving
